# Melaksanakan Sistem Pakar Haiwan

Contoh daripada [Kurikulum AI untuk Pemula](http://github.com/microsoft/ai-for-beginners).

Dalam contoh ini, kita akan melaksanakan sistem berasaskan pengetahuan yang mudah untuk menentukan haiwan berdasarkan beberapa ciri fizikal. Sistem ini boleh diwakili oleh pokok AND-OR berikut (ini adalah sebahagian daripada keseluruhan pokok, kita boleh menambah lebih banyak peraturan dengan mudah):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.ms.png)


## Sistem Pakar Kita Sendiri dengan Inferens Belakang

Mari kita cuba mendefinisikan satu bahasa mudah untuk perwakilan pengetahuan berdasarkan peraturan pengeluaran. Kita akan menggunakan kelas Python sebagai kata kunci untuk mendefinisikan peraturan. Terdapat tiga jenis kelas utama:
* `Ask` mewakili soalan yang perlu ditanya kepada pengguna. Ia mengandungi set jawapan yang mungkin.
* `If` mewakili satu peraturan, dan ia hanyalah sintaks ringkas untuk menyimpan kandungan peraturan tersebut.
* `AND`/`OR` adalah kelas untuk mewakili cabang AND/OR dalam pokok. Ia hanya menyimpan senarai argumen di dalamnya. Untuk mempermudahkan kod, semua fungsi didefinisikan dalam kelas induk `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Dalam sistem kami, memori kerja akan mengandungi senarai **fakta** sebagai **pasangan atribut-nilai**. Pangkalan pengetahuan boleh ditakrifkan sebagai satu kamus besar yang memetakan tindakan (fakta baru yang perlu dimasukkan ke dalam memori kerja) kepada syarat-syarat, yang dinyatakan sebagai ungkapan AND-OR. Selain itu, beberapa fakta boleh `Ask`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Untuk melaksanakan inferens ke belakang, kita akan mendefinisikan kelas `Knowledgebase`. Ia akan mengandungi:
* `memory` kerja - sebuah kamus yang memetakan atribut kepada nilai
* `rules` Knowledgebase dalam format seperti yang ditakrifkan di atas

Dua kaedah utama adalah:
* `get` untuk mendapatkan nilai atribut, melaksanakan inferens jika perlu. Sebagai contoh, `get('color')` akan mendapatkan nilai slot warna (ia akan bertanya jika perlu, dan menyimpan nilai untuk kegunaan kemudian dalam memori kerja). Jika kita bertanya `get('color:blue')`, ia akan bertanya tentang warna, dan kemudian mengembalikan nilai `y`/`n` bergantung pada warna.
* `eval` melaksanakan inferens sebenar, iaitu menelusuri pokok AND/OR, menilai sub-matlamat, dan sebagainya.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Sekarang mari kita tentukan pangkalan pengetahuan haiwan kita dan lakukan konsultasi. Perhatikan bahawa panggilan ini akan menanyakan soalan kepada anda. Anda boleh menjawab dengan menaip `y`/`n` untuk soalan ya-tidak, atau dengan menentukan nombor (0..N) untuk soalan dengan jawapan pelbagai pilihan yang lebih panjang.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Menggunakan PyKnow untuk Inferens Hadapan

Dalam contoh seterusnya, kita akan cuba melaksanakan inferens hadapan menggunakan salah satu perpustakaan untuk perwakilan pengetahuan, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** ialah perpustakaan untuk mencipta sistem inferens hadapan dalam Python, yang direka untuk menyerupai sistem lama klasik [CLIPS](http://www.clipsrules.net/index.html).

Kita juga boleh melaksanakan rantaian hadapan sendiri tanpa banyak masalah, tetapi pelaksanaan naif biasanya tidak begitu cekap. Untuk pemadanan peraturan yang lebih berkesan, algoritma khas [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) digunakan.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Kami akan mentakrifkan sistem kami sebagai kelas yang mewarisi `KnowledgeEngine`. Setiap peraturan ditakrifkan oleh fungsi berasingan dengan anotasi `@Rule`, yang menentukan bila peraturan tersebut harus dilaksanakan. Di dalam peraturan, kami boleh menambah fakta baharu menggunakan fungsi `declare`, dan penambahan fakta tersebut akan menyebabkan beberapa peraturan lain dipanggil oleh enjin inferens ke hadapan.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Sebaik sahaja kita telah mentakrifkan pangkalan pengetahuan, kita mengisi memori kerja kita dengan beberapa fakta awal, dan kemudian memanggil kaedah `run()` untuk melaksanakan inferens. Anda boleh melihat hasilnya bahawa fakta-fakta baru yang disimpulkan ditambah ke dalam memori kerja, termasuk fakta akhir tentang haiwan tersebut (jika kita menetapkan semua fakta awal dengan betul).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
